In [29]:
from sklearn import svm

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
clf = svm.SVC(kernel='linear')

In [33]:
cleaned_path = '/Users/pranaymishra/Desktop/ml_practice/ocean_dataset/cleaned'

In [34]:
dirty_img_path = '/Users/pranaymishra/Desktop/ml_practice/ocean_dataset/polluted'

In [35]:
import os
from skimage import io, color, feature

In [36]:
import os
import imageio
from skimage import color, feature

def load_images_and_labels(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder, filename)
            
            # Omit pilmode argument when using PyAV plugin
            img = imageio.imread(image_path)
            
            if img.shape[2] == 4:  # Check if image has 4 channels (RGBA)
                img = color.rgba2rgb(img)
            
            img = color.rgb2gray(img)
            hog_features = feature.hog(img)
            images.append(hog_features)
            labels.append(label)
    return images, labels


In [37]:
import numpy as np

In [38]:
polluted_images, polluted_labels = load_images_and_labels(cleaned_path, 1)
cleaned_images, cleaned_labels = load_images_and_labels(dirty_img_path, 0)

/var/folders/0d/srwhm0y53j14k9ypbh_kj7v40000gn/T/ipykernel_20253/2473183843.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image_path)


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Assuming all HOG features have the same length, let's say `hog_feature_length`
hog_feature_length = len(polluted_images[0])

# Pad or truncate HOG features to ensure a fixed length
def pad_or_truncate(features, target_length):
    return [feat[:target_length] if len(feat) >= target_length else np.pad(feat, (0, target_length - len(feat)), 'constant') for feat in features]

# Combine data and labels
all_images = np.vstack((pad_or_truncate(polluted_images, hog_feature_length), 
                        pad_or_truncate(cleaned_images, hog_feature_length)))
all_labels = np.hstack((polluted_labels, cleaned_labels))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Use a pipeline to standardize the data and train the SVM
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear'))
svm_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
confusion_mat = confusion_matrix(y_test, predictions)

print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_mat)


Accuracy: 0.8888888888888888
Confusion Matrix:
[[45  0]
 [ 6  3]]


In [40]:
import numpy as np

# Assuming all HOG features have the same length, let's say `hog_feature_length`
hog_feature_length = len(polluted_images[0])

# Pad or truncate HOG features to ensure a fixed length
def pad_or_truncate(features, target_length):
    return [feat[:target_length] if len(feat) >= target_length else np.pad(feat, (0, target_length - len(feat)), 'constant') for feat in features]

# Pad or truncate features to ensure consistent length
padded_polluted_images = pad_or_truncate(polluted_images, hog_feature_length)
padded_cleaned_images = pad_or_truncate(cleaned_images, hog_feature_length)

# Calculate mean and standard deviation for each feature
mean_polluted = np.mean(padded_polluted_images, axis=0)
std_polluted = np.std(padded_polluted_images, axis=0)

mean_cleaned = np.mean(padded_cleaned_images, axis=0)
std_cleaned = np.std(padded_cleaned_images, axis=0)

print("Mean (Polluted):", mean_polluted)
print("Standard Deviation (Polluted):", std_polluted)
print("Mean (Cleaned):", mean_cleaned)
print("Standard Deviation (Cleaned):", std_cleaned)


Mean (Polluted): [0.07357159 0.0278003  0.03820825 ... 0.04888614 0.03253084 0.03341326]
Standard Deviation (Polluted): [0.08344233 0.04968844 0.06615634 ... 0.07368243 0.065941   0.06824429]
Mean (Cleaned): [0.11285578 0.04422699 0.05737951 ... 0.07222779 0.05454952 0.05758679]
Standard Deviation (Cleaned): [0.0852906  0.0499645  0.06444214 ... 0.06884036 0.05755687 0.0629661 ]
